<a href="https://colab.research.google.com/github/MatteoOnger/NIAC_Project/blob/main/NIAC_PMS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **NIAC Project: Pacman Maze Solver**

*   **Authors:** Stefano Capelli, Matteo Onger
*   **Date:** March 2025

**Documentation**:
*   Gymnasium: [website](https://gymnasium.farama.org/)
*   Scallop & Scallopy: [paper](https://www.researchgate.net/publication/369945806_Scallop_A_Language_for_Neurosymbolic_Programming), [repository](https://github.com/scallop-lang/scallop), [website](https://www.scallop-lang.org/)
*   Pacman Maze game: [description](https://www.scallop-lang.org/ssnp24/index.html#section-17:~:text=2%3A%20PacMan%20Agent-,In,-this%20part%2C%20we)

**Notes**:
*   ...

In [ ]:
# download project repository
!git clone -b main https://github.com/MatteoOnger/NIAC_Project.git

# download Scallopy
!wget https://github.com/scallop-lang/scallop/releases/download/0.1.4/scallopy-0.1.4-cp39-cp39-manylinux_2_27_x86_64.whl

In [ ]:
# download and install Miniconda
%env PYTHONPATH=

!wget https://repo.anaconda.com/miniconda/Miniconda3-py311_25.1.1-2-Linux-x86_64.sh
!chmod +x /content/Miniconda3-py311_25.1.1-2-Linux-x86_64.sh
!bash Miniconda3-py311_25.1.1-2-Linux-x86_64.sh -b -f -p /usr/local

import sys
sys.path.append('/usr/local/lib/python3.11/site-packages/')

In [ ]:
# force creation of the virtual environment 'niac'
!conda env remove -n niac -y || true
!conda env create -f /content/NIAC_Project/environment.yaml -y
!conda env list

---

In [6]:
%%bash
source activate niac
python3

import sys
print(f"-----\nCurrent Python version: {sys.version}\n-----")

import scallopy
ctx = scallopy.ScallopContext()

ctx.add_relation("hello", str)
ctx.add_facts("hello", [("Hello World",)])
ctx.run()

print(list(ctx.relation("hello")))
quit()

-----
Current Python version: 3.9.21 | packaged by conda-forge | (main, Dec  5 2024, 13:51:40) 
[GCC 13.3.0]
-----
[('Hello World',)]
